In [1]:
import sys

sys.path.append("..")

from typing import Tuple, Iterable, Optional
import re
import torch


# Define The Config

In [2]:
BATCH_SIZE = 2
FINETUNE_LR = 1e-5
LR = 1e-4
EPOCHS = 30
TOTAL_STEPS = (42_000 // BATCH_SIZE) * EPOCHS
WARMUP_STEPS = int(TOTAL_STEPS * 0.1)
CONSTANT_STEPS = int(TOTAL_STEPS * 0.4)
WEIGHT_DECAY = 5e-3

CKPT_DIR = "ckpt"
LOG_DIR = "logs"

# Prepare Dataset

In [3]:
# Convert original dataset to WebDataset tar file for faster loading on Google Colab
from src.datamodule import VLSP2020Dataset, VLSP2020TarDataset

# from torch.utils.data import random_split

# dts = VLSP2020Dataset("../data/vlsp2020_train_set_02")
# train_set, val_set = random_split(dts, [42_000, 14_427])

# VLSP2020TarDataset("../data/vlsp2020_train_set.tar").convert(train_set)
# VLSP2020TarDataset("../data/vlsp2020_val_set.tar").convert(val_set)

/home/hoang/.local/lib/python3.10/site-packages/pkg_resources/__init__.py:123: PkgResourcesDeprecationWarning: 1.16.0-unknown is an invalid version and will not be supported in a future release
  warnings.warn(
/home/hoang/.local/lib/python3.10/site-packages/pkg_resources/__init__.py:123: PkgResourcesDeprecationWarning: 0.1.43ubuntu1 is an invalid version and will not be supported in a future release
  warnings.warn(
/home/hoang/.local/lib/python3.10/site-packages/pkg_resources/__init__.py:123: PkgResourcesDeprecationWarning: 1.1build1 is an invalid version and will not be supported in a future release
  warnings.warn(


## Get dataloader and preprocessing

In [4]:
train_dataset = VLSP2020TarDataset("../data/vlsp2020_train_set.tar").load()
val_dataset = VLSP2020TarDataset("../data/vlsp2020_val_set.tar").load()

In [5]:
import string

from src.datamodule.vlsp2020 import get_dataloader


def remove_punctuation(text: str):
    return text.translate(str.maketrans("", "", string.punctuation)).lower()


train_loader = get_dataloader(
    train_dataset,
    return_transcript=True,
    batch_size=BATCH_SIZE,
    target_transform=remove_punctuation,
)

val_loader = get_dataloader(
    val_dataset,
    return_transcript=True,
    batch_size=BATCH_SIZE,
    target_transform=remove_punctuation,
)


In [6]:
for batch in train_loader:
    print(batch)
    break

(('và hình như tôi cảm giác là qua từng vòng thi thì cái họng của bạn tốt hơn rồi thì phải', 'đến đây thì mọi bắt đầu thấy được cuộc chiến khốc liệt như thế nào rồi đúng không ạ họ rất là tính tay rất kĩ thưa quý vị chọn ô màu của mình mời em đọc câu tiếp theo'), (tensor([-0.0012,  0.0088,  0.0054,  ..., -0.0038, -0.0009,  0.0046]), tensor([-0.0760, -0.0811, -0.0728,  ..., -0.0032, -0.0034, -0.0035])))


# Metrics

In [7]:
def levenshtein_distance(source: Tuple[str], target: Tuple[str]):
    """
    Compute the Levenshtein distance between two sequences.
    """

    n, m = len(source), len(target)
    if n > m:
        # Make sure n <= m, to use O(min(n,m)) space
        source, target = target, source
        n, m = m, n

    current_row = range(n + 1)  # Keep current and previous row, not entire matrix
    for i in range(1, m + 1):
        previous_row, current_row = current_row, [i] + [0] * n
        for j in range(1, n + 1):
            add, delete, change = (
                previous_row[j] + 1,
                current_row[j - 1] + 1,
                previous_row[j - 1],
            )
            if source[j - 1] != target[i - 1]:
                change += 1
            current_row[j] = min(add, delete, change)

    distance = current_row[n]

    del current_row
    del previous_row

    return distance


def word_error_rate(prediction: str, transcript: str):
    pattern = r"\W+"

    prediction = re.split(pattern, prediction)
    transcript = re.split(pattern, transcript)

    return levenshtein_distance(prediction, transcript) / len(transcript)


def character_error_rate(prediction: str, transcript: str):
    return levenshtein_distance(prediction, transcript) / len(transcript)


# Prepare Tokenizer

In [8]:
import json
from torch.utils.data import DataLoader


class VocabBuilder:
    DELIM_TOKEN = "|"
    UNK_TOKEN = "<unk>"
    PAD_TOKEN = "<pad>"

    def __init__(self, vocab_file: str):
        self.vocab_file = vocab_file
        self._vocab_set = set()

    def add(self, texts: Iterable[str]):
        for t in texts:
            self._vocab_set.update(t)

    def build(self):
        self._vocab_dict = {c: i for i, c in enumerate(self._vocab_set)}

        # replace space with pipe for clearer visualization
        self._vocab_dict[self.DELIM_TOKEN] = self._vocab_dict[" "]
        del self._vocab_dict[" "]

        # add unknown token so that model can handle unseen characters
        self._vocab_dict[self.UNK_TOKEN] = len(self._vocab_dict)

        # add padding token for CTC
        self._vocab_dict[self.PAD_TOKEN] = len(self._vocab_dict)

        return self._vocab_dict

    @property
    def vocab_dict(self):
        return self._vocab_dict

    def save(self):
        with open(self.vocab_file, "w") as f:
            json.dump(self.vocab_dict, f, ensure_ascii=False)

    def load(self):
        with open(self.vocab_file, "r") as f:
            self._vocab_dict = json.load(f)

        return self._vocab_dict


# Speech Recognizer

## Define Model

In [9]:
from pytorch_lightning import LightningModule
from transformers import (
    Wav2Vec2ForPreTraining,
    Wav2Vec2CTCTokenizer,
    Wav2Vec2FeatureExtractor,
)
from torchmetrics import MeanMetric

from src.utils.scheduler import TriStateScheduler

class SpeechRecognizer(LightningModule):
    def __init__(
        self,
        wav2vec2: Wav2Vec2ForPreTraining,
        tokenizer: Wav2Vec2CTCTokenizer,
        feature_extractor: Wav2Vec2FeatureExtractor,
    ):
        super().__init__()


        self.hidden_size = wav2vec2.config.hidden_size
        self.vocab_size = tokenizer.vocab_size

        self.wav2vec2 = wav2vec2
        self.wav2vec2.freeze_feature_extractor()
        self.tokenizer = tokenizer
        self.feature_extractor = feature_extractor

        self.dropout = torch.nn.Dropout(0.1)
        self.fc = torch.nn.Sequential(
            torch.nn.Linear(self.hidden_size, self.hidden_size // 2),
            torch.nn.ReLU(inplace=True),
            torch.nn.Linear(self.hidden_size // 2, self.vocab_size),
        )

        self.criterion = torch.nn.CTCLoss(blank=tokenizer.pad_token_id)

        self.train_loss = MeanMetric()

    def forward(self, waveforms: Tuple[torch.Tensor], transcripts: Tuple[str] = None):
        # convert torch.Tensor to numpy.ndarray
        waveforms = tuple(waveform.cpu().numpy() for waveform in waveforms)

        extracted = self.feature_extractor(
            waveforms,
            sampling_rate=16000,
            padding=True,
            return_tensors="pt",
            return_attention_mask=True,
        )

        outputs = self.wav2vec2(
            extracted.input_values,
            attention_mask=extracted.attention_mask,
        )

        # hidden_states.shape == (batch_size, sequence_length, hidden_size)
        hidden_states = outputs[0]
        hidden_states = self.dropout(hidden_states)

        # logits.shape == (batch_size, sequence_length, vocab_size)
        logits = self.fc(hidden_states)

        if transcripts is not None:
            # get the length of valids sequence
            input_lengths = self.wav2vec2._get_feat_extract_output_lengths(
                extracted.attention_mask.sum(-1)
            ).to(torch.long)

            # tokenize transcripts
            target_ids, target_lengths = self.tokenizer(
                transcripts, padding=True, return_length=True, return_tensors="pt"
            ).values()

            # (batch_size, sequence_length, vocab_size) -> (sequence_length, batch_size, vocab_size)
            log_probs = torch.nn.functional.log_softmax(logits, dim=-1).transpose_(0, 1)

            # compute loss
            loss = self.criterion(log_probs, target_ids, input_lengths, target_lengths)

            return loss, logits
        else:
            return logits

    def training_step(self, batch, batch_idx):
        transcripts, waveforms = batch

        loss, logits = self(waveforms, transcripts)

        self.train_loss(loss)

        if batch_idx % 100 == 0:
            self.log("train/loss", self.train_loss, on_step=True, on_epoch=True)

        return loss

    def on_train_epoch_end(self) -> None:
        self.train_loss.reset()

    def validation_step(self, batch, batch_idx):
        transcripts, waveforms = batch

        logits = self(waveforms)

        predicted_ids = torch.argmax(logits, dim=-1).cpu().numpy()

        predicted_texts = self.tokenizer.batch_decode(predicted_ids)

        wer = word_error_rate(predicted_texts, transcripts)
        cer = character_error_rate(predicted_texts, transcripts)

        self.log("val/wer", wer, on_epoch=True)
        self.log("val/cer", cer, on_epoch=True)

        return wer, cer

    def configure_optimizers(self):

        optimizer = torch.optim.AdamW([
            {"params": self.wav2vec2.parameters(), "lr": FINETUNE_LR},
            {"params": self.fc.parameters(), "lr": LR},
        ], lr=LR, weight_decay=WEIGHT_DECAY
        )

        scheduler = TriStateScheduler(
            optimizer,
            total_steps=TOTAL_STEPS,
            warmup_steps=WARMUP_STEPS,
            constant_steps=CONSTANT_STEPS,
            factor=1e-3
        )

        return {
            "optimizer": optimizer,
            "lr_scheduler": {
                "scheduler": scheduler,
                "interval": "step",
                "frequency": 1,
            }
        }


# Training

In [10]:
model_name = "nguyenvulebinh/wav2vec2-base-vietnamese-250h"

wav2vec2 = Wav2Vec2ForPreTraining.from_pretrained(model_name)
tokenizer = Wav2Vec2CTCTokenizer.from_pretrained(model_name)
feature_extractor = Wav2Vec2FeatureExtractor.from_pretrained(model_name)

/home/hoang/.local/lib/python3.10/site-packages/transformers/configuration_utils.py:369: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  warnings.warn(
Some weights of the model checkpoint at nguyenvulebinh/wav2vec2-base-vietnamese-250h were not used when initializing Wav2Vec2ForPreTraining: ['lm_head.weight', 'lm_head.bias']
- This IS expected if you are initializing Wav2Vec2ForPreTraining from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2ForPreTraining from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model

In [16]:
tokenizer.vocab_size

110

In [18]:
tokenizer.get_vocab()

{'ẻ': 0,
 '6': 1,
 'ụ': 2,
 'í': 3,
 '3': 4,
 'ỹ': 5,
 'ý': 6,
 'ẩ': 7,
 'ở': 8,
 'ề': 9,
 'õ': 10,
 '7': 11,
 'ê': 12,
 'ứ': 13,
 'ỏ': 14,
 'v': 15,
 'ỷ': 16,
 'a': 17,
 'l': 18,
 'ự': 19,
 'q': 20,
 'ờ': 21,
 'j': 22,
 'ố': 23,
 'à': 24,
 'ỗ': 25,
 'n': 26,
 'é': 27,
 'ủ': 28,
 'у': 29,
 'ô': 30,
 'u': 31,
 'y': 32,
 'ằ': 33,
 '4': 34,
 'w': 35,
 'b': 36,
 'ệ': 37,
 'ễ': 38,
 's': 39,
 'ì': 40,
 'ầ': 41,
 'ỵ': 42,
 '8': 43,
 'd': 44,
 'ể': 45,
 'r': 47,
 'ũ': 48,
 'c': 49,
 'ạ': 50,
 '9': 51,
 'ế': 52,
 'ù': 53,
 'ỡ': 54,
 '2': 55,
 't': 56,
 'i': 57,
 'g': 58,
 '́': 59,
 'ử': 60,
 '̀': 61,
 'á': 62,
 '0': 63,
 'ậ': 64,
 'e': 65,
 'ộ': 66,
 'm': 67,
 'ẳ': 68,
 'ợ': 69,
 'ĩ': 70,
 'h': 71,
 'â': 72,
 'ú': 73,
 'ọ': 74,
 'ồ': 75,
 'ặ': 76,
 'f': 77,
 'ữ': 78,
 'ắ': 79,
 'ỳ': 80,
 'x': 81,
 'ó': 82,
 'ã': 83,
 'ổ': 84,
 'ị': 85,
 '̣': 86,
 'z': 87,
 'ả': 88,
 'đ': 89,
 'è': 90,
 'ừ': 91,
 'ò': 92,
 'ẵ': 93,
 '1': 94,
 'ơ': 95,
 'k': 96,
 'ẫ': 97,
 'p': 98,
 'ấ': 99,
 'ẽ': 100,
 'ỉ': 101

In [11]:
model = SpeechRecognizer(wav2vec2, tokenizer, feature_extractor)

/home/hoang/.local/lib/python3.10/site-packages/transformers/models/wav2vec2/modeling_wav2vec2.py:1362: FutureWarning: The method `freeze_feature_extractor` is deprecated and will be removed in Transformers v5.Please use the equivalent `freeze_feature_encoder` method instead.
  warnings.warn(


In [12]:
from pytorch_lightning import Trainer
from pytorch_lightning.callbacks import ModelCheckpoint
from pytorch_lightning.loggers import TensorBoardLogger

trainer = Trainer(
    accelerator="cpu",
    callbacks=[
        ModelCheckpoint(CKPT_DIR, monitor="val/wer", mode="min", save_top_k=1)
    ],
    logger=TensorBoardLogger(LOG_DIR),
    max_epochs=EPOCHS
)

GPU available: True (cuda), used: False
/home/hoang/.local/lib/python3.10/site-packages/pkg_resources/__init__.py:123: PkgResourcesDeprecationWarning: 1.16.0-unknown is an invalid version and will not be supported in a future release
  warnings.warn(
/home/hoang/.local/lib/python3.10/site-packages/pkg_resources/__init__.py:123: PkgResourcesDeprecationWarning: 0.1.43ubuntu1 is an invalid version and will not be supported in a future release
  warnings.warn(
/home/hoang/.local/lib/python3.10/site-packages/pkg_resources/__init__.py:123: PkgResourcesDeprecationWarning: 1.1build1 is an invalid version and will not be supported in a future release
  warnings.warn(
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/home/hoang/.local/lib/python3.10/site-packages/pytorch_lightning/trainer/setup.py:175: PossibleUserWarning: GPU available but not used. Set `accelerator` and `devices` using `Trainer(accelerator='gpu', devices=1)`.
  ra

In [13]:
trainer.fit(model, train_loader, val_loader)

Missing logger folder: logs/lightning_logs

  | Name       | Type                   | Params
------------------------------------------------------
0 | wav2vec2   | Wav2Vec2ForPreTraining | 95.0 M
1 | dropout    | Dropout                | 0     
2 | fc         | Sequential             | 337 K 
3 | criterion  | CTCLoss                | 0     
4 | train_loss | MeanMetric             | 0     
------------------------------------------------------
91.2 M    Trainable params
4.2 M     Non-trainable params
95.4 M    Total params
381.529   Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

/home/hoang/.local/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:224: PossibleUserWarning: The dataloader, val_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 8 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(

KeyboardInterrupt



In [ ]:
tokenizer(["xin chào 😃", "cô gái nông thôn"], return_tensors="pt", padding=True, return_length=True).input_ids.shape

torch.Size([2, 16])

In [ ]:
input_ids

tensor([[ 81,  57,  26,  46,  49,  71,  24, 105,  46, 108, 109, 109, 109, 109,
         109, 109],
        [ 49,  30,  46,  58,  62,  57,  46,  26,  30,  26,  58,  46,  56,  71,
          30,  26]])

In [ ]:
attention_mask

tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])

In [ ]:
tokenizer.batch_decode(ids)

['xin chào <unk>', 'cô gái nông thôn']

In [ ]:
tokenizer.get_vocab()

{'ẻ': 0,
 '6': 1,
 'ụ': 2,
 'í': 3,
 '3': 4,
 'ỹ': 5,
 'ý': 6,
 'ẩ': 7,
 'ở': 8,
 'ề': 9,
 'õ': 10,
 '7': 11,
 'ê': 12,
 'ứ': 13,
 'ỏ': 14,
 'v': 15,
 'ỷ': 16,
 'a': 17,
 'l': 18,
 'ự': 19,
 'q': 20,
 'ờ': 21,
 'j': 22,
 'ố': 23,
 'à': 24,
 'ỗ': 25,
 'n': 26,
 'é': 27,
 'ủ': 28,
 'у': 29,
 'ô': 30,
 'u': 31,
 'y': 32,
 'ằ': 33,
 '4': 34,
 'w': 35,
 'b': 36,
 'ệ': 37,
 'ễ': 38,
 's': 39,
 'ì': 40,
 'ầ': 41,
 'ỵ': 42,
 '8': 43,
 'd': 44,
 'ể': 45,
 'r': 47,
 'ũ': 48,
 'c': 49,
 'ạ': 50,
 '9': 51,
 'ế': 52,
 'ù': 53,
 'ỡ': 54,
 '2': 55,
 't': 56,
 'i': 57,
 'g': 58,
 '́': 59,
 'ử': 60,
 '̀': 61,
 'á': 62,
 '0': 63,
 'ậ': 64,
 'e': 65,
 'ộ': 66,
 'm': 67,
 'ẳ': 68,
 'ợ': 69,
 'ĩ': 70,
 'h': 71,
 'â': 72,
 'ú': 73,
 'ọ': 74,
 'ồ': 75,
 'ặ': 76,
 'f': 77,
 'ữ': 78,
 'ắ': 79,
 'ỳ': 80,
 'x': 81,
 'ó': 82,
 'ã': 83,
 'ổ': 84,
 'ị': 85,
 '̣': 86,
 'z': 87,
 'ả': 88,
 'đ': 89,
 'è': 90,
 'ừ': 91,
 'ò': 92,
 'ẵ': 93,
 '1': 94,
 'ơ': 95,
 'k': 96,
 'ẫ': 97,
 'p': 98,
 'ấ': 99,
 'ẽ': 100,
 'ỉ': 101

# Inference

In [ ]:
from src.model.modules import Wav2Vec2Processor
from src.model import Wav2Vec2PretrainingModule
import torch


def speech_to_text(waveforms: Tuple[torch.Tensor, ...]):

    batched_waveforms, wavelengths = Wav2Vec2Processor()(waveforms)
    attention_masks = Wav2Vec2PretrainingModule._compute_attention_mask(wavelengths)

    logits = model(batched_waveforms, attention_mask=attention_masks).logits
    predicted_ids = torch.argmax(logits, dim=-1)

    return processor.batch_decode(predicted_ids)


In [ ]:
cer, wer = 0, 0
n_items = 0

for batch_idx, batch in enumerate(dataloader):
    transcripts, waveforms = batch
    predicted_transcripts = speech_to_text(waveforms)

    for predicted_transcript, transcript in zip(predicted_transcripts, transcripts):
        cer += character_error_rate(predicted_transcript, transcript)
        wer += word_error_rate(predicted_transcript, transcript)

        n_items += 1

    if batch_idx > 100:
        break

    del transcript
    del waveforms
    del predicted_transcript
    del batch

cer /= n_items
wer /= n_items

print(f"Character error rate: {cer:.2%}")
print(f"Word error rate: {wer:.2%}")


Character error rate: 25.47%
Word error rate: 32.33%


In [ ]:
import pandas as pd

m = []

for batch_idx, batch in enumerate(dataloader):
    if batch_idx > 5:
        break

    transcripts, waveforms = batch
    predicted_transcripts = speech_to_text(waveforms)

    for predicted_transcript, transcript in zip(predicted_transcripts, transcripts):
        m.append(
            {
                "transcript": transcript,
                "predicted_transcript": predicted_transcript,
            }
        )


pd.set_option("display.max_colwidth", None)
pd.DataFrame.from_dict(m)


,transcript,predicted_transcript
0,trước khi bắt đầu khởi hành hiển dự định đạp xe vào bốn giờ ba mươi phút sáng hàng ngày,trước khi bắt đầu khởi hành hiện dự định đạp xe và bốn giờ ba mươi phút sáng hằng ngày
1,<unk> ngon,ớnđyvn
2,xin được cảm ơn những chia sẻ đầy cảm xúc của các khách mời đã giúp cho chúng tôi những người thuộc nhiều thế hệ được lớn lên trong hòa bình thêm trân trọng những mất mát hi sinh của thế hệ đi trước để có được một việt nam hòa bình giúp cho chúng tôi,xin được cảm ơn những chia sẻ đầy cảm xúc của các khách mời đã giúp cho chúng tôi những người thuộc nhiều thế hệ được lớn lên trong hòa bình thêm trân trọng những mất mát hi sinh của thấy đi trước để có được một việt nam hòa bình giúp cho chúng tôi
3,nguyễn trang đem thi thể chúa trịnh nộp cho quân tây sơn,nguyễn trang đem thi thể chúa trịnh nộp cho quân tây sơn
4,theo đó chủ mưu trong vụ án là bị cáo nguyễn minh hùng bị tuyên mười bảy năm tù với vai trò chỉ,theo đó chủ mưu trong vụ án là bị cáo nguyễn minh hùng bị tuyên mười bảy năm tù với vai trò chỉ
5,bây giờ sẽ là phần thi cuối cùng và cũng là <unk>,bây giờ sẽ là phần thi cuối cùng và cũn là
6,đồng thời lại nghe theo kiến nghị của quân sư ông cho đúc trở lại tiền ngũ thù khôi phục lưu thông tiền tệ trong địa bàn đem đến tiện lợi cho sinh hoạt của nhân dân không còn phải dùng xếp vải làm phương tiện thanh toán,đồng thời lại nghe theo kýn nghị của quân sư ông cho đúc trở lại tiền ngủ thù khôi phục lưu thông tiền tệ trong địa bàn đem đến tiền lợi cho sinh hoạt của nhân dân không còn phải dùng xếp vãi làm phương tiện thanh toán
7,chính quyền của tổng thống trăm nhằm cắt giảm các chương trình trợ cấp an sinh xã hội,chính quyền của tổng thống trămp nhắm cắt giảm các chương trình trơa cấp an sinh xã hộicc
8,văn học việt nam là sự tích hợp từ hai dòng văn học dân gian và văn học viết của những người dùng tiếng việt,văn học việt nam là sự tích hợp từ hai dòng văn học dân gian và văn học viết của những người dùng tiếng việt
9,ui tình,ừừừ
